# Week 36

1. Explore the dataset from https://huggingface.co/datasets/coastalcph/tydi_xor_rc. Familiarize yourself with the dataset card, download the
dataset and explore its columns. Summarize basic data statistics for training and validation data in each of the languages Finnish (fi), Japanese
(ja) and Russian (ru). # Sofia + Tom

2. For each of the languages Finnish, Japanese and Russian, report the 5
most common words in the questions from the training set. What kind of
words are they?

3. Implement a rule-based classifier that predicts whether a question is answerable or impossible, only using the document (context) and question.
You may use machine translation as a component. Use the answerable
field to evaluate it on the validation set. What is the performance of your
classifier for each of the languages Finnish, Japanese and Russian?



####  1. Exploring the dataset

In [ ]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
print(df.head)

<bound method NDFrame.head of                                                 question  \
0      উইকিলিকস কত সালে সর্বপ্রথম ইন্টারনেটে প্রথম তথ...   
1               দ্বিতীয় বিশ্বযুদ্ধে কোন দেশ পরাজিত হয় ?   
2      মার্কিন যুক্তরাষ্ট্রের সংবিধান অনুযায়ী মার্কিন...   
3      আরব-ইসরায়েলি যুদ্ধে আরবের মোট কয়জন সৈন্যের মৃ...   
4              বিশ্বে প্রথম পুঁজিবাদী সমাজ কবে গড়ে ওঠে ?   
...                                                  ...   
15321              కోళ్లు ఎక్కువగా ఏ దేశంలో కనిపిస్తాయి?   
15322       క్షయ వ్యాధికి విరుగుడు ఏ దేశంలో కనుగొన్నారు?   
15323                 ఖురాన్ ఏ అరబ్బీ భాషలో ఎవరు రాసారు?   
15324  టెక్సస్ రాష్ట్రంలోని అతిపెద్ద మానవ నిర్మితం ఏది ?   
15325         తమిళనాడులో రాష్ట్ర మొదటి ముఖ్యమంత్రి ఎవరు?   

                                                 context lang  answerable  \
0      WikiLeaks () is an international non-profit or...   bn        True   
1      The war in Europe concluded with an invasion o...   bn        True   
2      Same-sex ma

In [ ]:
df.describe()

,answer_start
count,15326.000000
mean,157.062769
std,226.748482
min,-1.000000
25%,13.000000
50%,78.000000
75%,210.000000
max,3964.000000


In [ ]:
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Explore the dataset columns (training and validation)
print("Training Data Columns:", train_df.columns)
print("Validation Data Columns:", val_df.columns)

Training Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')
Validation Data Columns: Index(['question', 'context', 'lang', 'answerable', 'answer_start', 'answer',
       'answer_inlang'],
      dtype='object')


In [ ]:
finnish_questions = df[df['lang'] == 'fi']
japanese_questions = df[df['lang'] == 'ja']
russian_questions = df[df['lang'] == 'ru']

# Summarize basic statistics
print("Statistics for Finnish Questions:")
print(finnish_questions.describe(include='all'))  # or include only numeric features by default
print("\nStatistics for Japanese Questions:")
print(japanese_questions.describe(include='all'))
print("\nStatistics for Russian Questions:")
print(russian_questions.describe(include='all'))

Statistics for Finnish Questions:
                                                question  \
count                                               2126   
unique                                              2100   
top     Kuinka monta peliä Final Fantasy-sarjaan kuuluu?   
freq                                                   3   
mean                                                 NaN   
std                                                  NaN   
min                                                  NaN   
25%                                                  NaN   
50%                                                  NaN   
75%                                                  NaN   
max                                                  NaN   

                                                  context  lang answerable  \
count                                                2126  2126       2126   
unique                                               2054     1          2   
top     Mir

We can see, that there are 2301 Japanese questions (2100 unique), 2126 Finnish questions (1743 unique) and 1983 Russian questions (1612 unique).

#### 2.  5 most common words

In [ ]:
!pip install googletrans==4.0.0-rc1

In [ ]:
from collections import Counter
from googletrans import Translator
translator = Translator()

In [ ]:
# Find 5 most common words in Finnish in df dataset

finnish_questions = df[df['lang'] == 'fi']['question']

# Combine all questions into a single string
all_questions_text = ' '.join(finnish_questions.astype(str))

# Tokenize the text
words = all_questions_text.lower().split()

# Count the frequency of each word
word_counts = Counter(words)

# Get the 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='fi', dest='en').text  # Translate word from Finnish to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Finnish questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

# print("Top 5 most common words in Finnish questions:", top_5_words)


Top 5 most common words in Finnish questions with English translations:
on (count: 582) - English: there is
mikä (count: 328) - English: What
milloin (count: 287) - English: When
vuonna (count: 227) - English: in
kuka (count: 215) - English: Who


In [ ]:
# Find 5 most common words in Russian in df dataset
russian_questions = df[df['lang'] == 'ru']['question']

all_questions_text = ' '.join(russian_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ru', dest='en').text  # Translate word from Russian to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Russian questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

#print("Top 5 most common words in Russian questions:", top_5_words)




Top 5 most common words in Russian questions with English translations:
в (count: 978) - English: V
сколько (count: 426) - English: How many
на (count: 385) - English: on
когда (count: 251) - English: When
кто (count: 209) - English: Who


For both Finnish and Russian it appears the the most common words are question
words or "connecting"/grammar words.

Let's proceed to Japanese:

In [ ]:
# Find 5 most common words in Japanese in df dataset
japanese_questions = df[df['lang'] == 'ja']['question']

all_questions_text = ' '.join(japanese_questions.astype(str))

words = all_questions_text.lower().split()

word_counts = Counter(words)

top_5_words = word_counts.most_common(5)

print("Top 5 most common words in Japanese questions:", top_5_words)

Top 5 most common words in Japanese questions: [('pygが最初に発表した曲は何', 3), ('オーロラ号の漂流時、オーロラ号に乗組員は何人いた？', 3), ('カンザスシティが設立されたのはいつ', 3), ('クロイツ群が初めて発見されたのはいつ', 3), ('ライド', 2)]


We can see that instead of actually counting words in japanese, it counted whole sentences. We'll try to use a specific tokenizer for japanese:

In [ ]:
# Load tokenizer directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_ja = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
#model_ja = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

tokenizer_fi = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-fi-en")
#model_fi = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-fi-en")

tokenizer_ru = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en")
#model_ru = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")



/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize_japanese_sentences(sentence):
    tokens = tokenizer_ja.tokenize(sentence)
    return " ".join(tokens)

japanese_questions = df[df['lang'] == 'ja']['question']

# Apply tokenization to the 'japanese_questions' series
tokenized_japanese_questions = japanese_questions.apply(tokenize_japanese_sentences)

# Print the original and tokenized questions
for original, tokenized in zip(japanese_questions.head(5), tokenized_japanese_questions.head(5)):
    print(f"Original: {original}")
    print(f"Tokenized: {tokenized}")
    print("-" * 50)

Original: ポーランドで農地改革が行われたことがある？
Tokenized: ▁ポーランド で 農 地 改革 が 行わ れた こと がある ?
--------------------------------------------------
Original: ビスカヤ県で初めて進出した大規模鉱業会社は何？
Tokenized: ▁ビ スカ ヤ 県 で 初めて 進 出 した 大 規模 鉱 業 会社 は 何 ?
--------------------------------------------------
Original: 古代ローマ帝国はいつ起きた？
Tokenized: ▁古代 ローマ 帝国 は いつ 起き た ?
--------------------------------------------------
Original: スペイン・バスク州の州都はどこですか？
Tokenized: ▁スペイン ・ バス ク 州 の 州 都 は どこ です か ?
--------------------------------------------------
Original: イタリア王国海軍は第一次世界大戦中に何隻戦艦をつくった？
Tokenized: ▁イタリア 王国 海軍 は 第一 次 世界 大戦 中 に 何 隻 戦艦 を つく った ?
--------------------------------------------------


Now that the Japanese questions are properly tokenized, we can proceed to find the most common words:

In [ ]:
import re

def clean_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)  # Removes non-word characters (punctuation)
    return text.lower().split()

# Convert tokenized questions into a single string
all_questions_text = ' '.join(tokenized_japanese_questions.astype(str))

# Clean the text by removing punctuation
cleaned_words = clean_text(all_questions_text)

# Count word frequencies
word_counts = Counter(cleaned_words)

# Get the top 5 most common words
top_5_words = word_counts.most_common(5)

# Translate words dynamically
translated_top_5 = []
for word, count in top_5_words:
    translation = translator.translate(word, src='ja', dest='en').text  # Translate word from Japanese to English
    translated_top_5.append((word, count, translation))

# Print the top 5 words with translations
print("Top 5 most common words in Japanese questions with English translations:")
for word, count, translation in translated_top_5:
    print(f"{word} (count: {count}) - English: {translation}")

Top 5 most common words in Japanese questions with English translations:
は (count: 2224) - English: teeth
の (count: 1632) - English: of
何 (count: 542) - English: what
した (count: 469) - English: did
いつ (count: 459) - English: when


As we can see, the filtered Japanese results are similar to the Finnish and Russian results.

Note the "は" token. Altough it can be translated to "teeth", it is most commonly used as grammatical particle - used to indicate the subject of the sentence. It is a frequently used particle which explains why we see it so much.

In [ ]:
def clean_text(text):
    # Remove punctuation using regex
    text = re.sub(r'[^\w\s]', '', text)  # Removes non-word characters (punctuation)
    return text.lower().split()

def find_most_common_words(tokenizer, language, num_words=5):
    # Filter questions for the specified language
    questions_ = df[df['lang'] == language]['question']

    # Tokenize questions
    tokenized_questions = questions_.apply(lambda x: " ".join(tokenizer.tokenize(x)))

    # Convert tokenized questions into a single string
    all_questions_text = ' '.join(tokenized_questions.astype(str))

    # Clean the text by removing punctuation
    cleaned_words = clean_text(all_questions_text)

    # Count word frequencies
    word_counts = Counter(cleaned_words)

    # Get the top n most common words
    top_n_words = word_counts.most_common(num_words)

    # Translate words dynamically
    translated_top_n = []
    for word, count in top_n_words:
        translation = translator.translate(word, src=language, dest='en').text  # Translate word to English
        translated_top_n.append((word, count, translation))

    # Print the top n words with translations
    for word, count, translation in translated_top_n:
        print(f"{word} (count: {count}) - English: {translation}")

print("Top 5 most common words in Finnish questions:")
find_most_common_words(tokenizer_fi, 'fi')

print("\nTop 5 most common words in Japanese questions:")
find_most_common_words(tokenizer_ja, 'ja')

print("\nTop 5 most common words in Russian questions:")
find_most_common_words(tokenizer_ru, 'ru')

Top 5 most common words in Finnish questions:
on (count: 678) - English: there is
n (count: 420) - English: of
mikä (count: 328) - English: What
milloin (count: 287) - English: When
a (count: 242) - English: a

Top 5 most common words in Japanese questions:
は (count: 2224) - English: teeth
の (count: 1632) - English: of
何 (count: 542) - English: what
した (count: 469) - English: did
いつ (count: 459) - English: when

Top 5 most common words in Russian questions:
в (count: 1172) - English: V
на (count: 493) - English: on
а (count: 451) - English: A
сколько (count: 426) - English: How many
е (count: 317) - English: e


####  3. Rule-based Classifier

Classifier that predicts whether a question is answerable or impossible, only using the document (context) and question. You may use machine translation as a component.

In [ ]:
!pip install tqdm

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from sklearn.metrics import accuracy_score, confusion_matrix
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from tqdm import tqdm  # Import tqdm for progress bars

nltk.download('stopwords')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

def clean_tokens(tokens):
    # Remove punctuation tokens and stopwords, and change words to "basic" form (lemmatize)
    cleaned_tokens = [lemmatizer.lemmatize(token) for token in tokens if token.isalpha() and token not in stop_words]
    return cleaned_tokens

def predict_answerability(context, question, lang):
    translated_question = translator.translate(question, src=lang, dest='en').text

    # Tokenize both the context and the translated question
    tokenized_context = tokenizer.tokenize(context.lower())
    tokenized_question = tokenizer.tokenize(translated_question.lower())

    # Clean tokens of common words and punctuation
    cleaned_context = clean_tokens(tokenized_context)
    cleaned_question = clean_tokens(tokenized_question)

    # Check for overlap
    for token in cleaned_question:
        if token in cleaned_context:
            return 'True', translated_question

    # If there is no overlap, we determine the question unanswerable
    return 'False', translated_question


# Evaluate the classifier on the validation set
def evalute_classifier(df_, num_samples=100, seed=420):
  predictions = []
  true_labels = []

  sampled_rows = df_.sample(n=num_samples, random_state=seed)

  for index, row in tqdm(sampled_rows.iterrows(), total=num_samples):
      context = row['context']  # Always in English
      question = row['question']  # Could be in any language
      lang = row['lang']

      prediction, translated_question = predict_answerability(context, question, lang)

      predictions.append(1 if prediction == 'True' else 0)
      true_labels.append(1 if str(row['answerable']) == 'True' else 0)

      """print(f"Context: {context}")
      print(f"Question: {question}")
      print(f"Translated Question: {translated_question}")
      print(f"Prediction: {prediction}")  # This will be 'answerable' or 'impossible'
      print(f"True Label: {row['answerable']}")
      print("-" * 50)"""

  # Calculate accuracy
  accuracy = accuracy_score(true_labels, predictions)
  print(f"Accuracy: {accuracy * 100:.2f}%")

  # Calculate FP and FN percentages
  cm = confusion_matrix(true_labels, predictions, labels=[1, 0])
  tn, fp, fn, tp = cm.ravel()
  false_positive_percentage = (fp / (fp + tn)) * 100 if (fp + tn) > 0 else 0
  false_negative_percentage = (fn / (fn + tp)) * 100 if (fn + tp) > 0 else 0

  print(f"False Positive Percentage: {false_positive_percentage:.2f}%")
  print(f"False Negative Percentage: {false_negative_percentage:.2f}%")


print(f"Classifer evaluation for Finnish:")
evalute_classifier(val_df[val_df['lang'] == 'fi'])
print("-" * 50)
print(f"Classifer evaluation for Japanese:")
evalute_classifier(val_df[val_df['lang'] == 'ja'])
print("-" * 50)
print(f"Classifer evaluation for Russian:")
evalute_classifier(val_df[val_df['lang'] == 'ru'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Classifer evaluation for Finnish:


100%|██████████| 100/100 [01:43<00:00,  1.04s/it]


Accuracy: 68.00%
False Positive Percentage: 1.49%
False Negative Percentage: 93.94%
--------------------------------------------------
Classifer evaluation for Japanese:


 12%|█▏        | 12/100 [00:13<01:38,  1.11s/it]


KeyboardInterrupt: 

In [ ]:
# Just some attempts to connect the project with the gitub repo


#!git clone https://github.com/sofi388/nlp_ucph.git

In [ ]:
#%cd nlp_ucph

In [ ]:
#%cd /content/nlp_ucph

In [ ]:
#!find /content -name "NLP_Project.ipynb"

# Week 37

Let k be the number of members in your group (k ∈ {1, **2**, 3}). Implement
k different * language models for the questions in the three languages Finnish,
Japanese and Russian, as well as for the document contexts in English (total
**2 × 4** language models), using the training data. Evaluate each of them on the
validation data, report their performance and discuss the results. Reminder: a
language model is a function that takes text as input and returns its probability.

\* Different approach (n-gram/neural) or different n, different smoothing etc.
2


In [ ]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

**How to implement N-gram model:**

Step 1: Tokenization
Tokenize your text data into words or characters, depending on the N-gram size you choose (e.g., bigrams, trigrams).

Step 2: Build N-gram Frequency Tables
Create N-grams: Extract N-grams from the tokenized text.

For bigrams, create pairs of consecutive tokens.
For trigrams, create triplets of consecutive tokens.
Count Frequencies: Count the occurrences of each N-gram in your dataset.

Step 3: Compute Probabilities
Calculate Probabilities: Compute the probability of each N-gram by dividing its frequency by the frequency of its prefix (for bigrams) or the frequency of its prefix (for trigrams).

Step 4: Generate Text
Text Generation: Use the computed probabilities to generate text. Start with a seed word and iteratively add the most likely subsequent word based on the N-gram probabilities.

Repeat the above process for each language: Finnish, Japanese, Russian.

Tokenize the text using language-specific tools if necessary (see week 36).
Build N-gram models and compute probabilities.

 ####  1. Trigram model (Sofia)

In [ ]:
from collections import defaultdict, Counter
import math

class TrigramModel:
    def __init__(self, tokenizer, smoothing_coef):
        # Initialize the trigram, bigram, and unigram counts
        self.trigram_counts = defaultdict(Counter)
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.known_words = set()
        self.tokenizer = tokenizer
        self.smoothing_coef = smoothing_coef

    def train(self, sentences):
        """
        Train the trigram model on a list of tokenized sentences.
        :param sentences: List of sentences where each sentence is a list of words.
        """
        tokenized_sentences = [self.tokenizer.tokenize(sentence.lower()) for sentence in sentences]

        for sentence in tokenized_sentences:
            # Replace first occurrences with 'OOV' and update known_words set
            for i in range(len(sentence)):
                word = sentence[i]
                if word not in self.known_words:
                    sentence[i] = 'OOV'
                    self.known_words.add(word)

            # Count unigrams, bigrams, and trigrams
            for i in range(len(sentence) - 2):
                self.unigram_counts[sentence[i]] += 1
                self.bigram_counts[sentence[i]][sentence[i + 1]] += 1
                self.trigram_counts[(sentence[i], sentence[i + 1])][sentence[i + 2]] += 1

            # Count remaining bigram and unigram
            if len(sentence) > 1:
                self.unigram_counts[sentence[-2]] += 1
                self.bigram_counts[sentence[-2]][sentence[-1]] += 1
            self.unigram_counts[sentence[-1]] += 1  # Count the last word as a unigram

    def unigram_probability(self, word):
        """
        Calculate the unigram probability P(word).
        :param word: The word for which to calculate the probability.
        :return: The probability of the word.
        """
        total_count = sum(self.unigram_counts.values())
        vocab_size = len(self.known_words)  # Unique words
        if word in self.unigram_counts:
            return self.unigram_counts[word] / total_count
        else:
            return self.unigram_counts['OOV'] / total_count if 'OOV' in self.unigram_counts else 0

    def bigram_probability(self, w1, w2):
        """
        Calculate the bigram probability P(w2 | w1).
        :param w1: Previous word.
        :param w2: Current word.
        :return: Probability of w2 given w1.
        """
        vocab_size = len(self.known_words)
        if self.unigram_counts[w1] > 0:
            return (self.bigram_counts[w1][w2] + self.smoothing_coef) / (self.unigram_counts[w1] + self.smoothing_coef * vocab_size)
        else:
            return (self.bigram_counts['OOV'][w2] + self.smoothing_coef) / (self.unigram_counts['OOV'] + self.smoothing_coef * vocab_size)

    def trigram_probability(self, w1, w2, w3):
        """
        Calculate the trigram probability P(w3 | w1, w2).
        :param w1: First word.
        :param w2: Second word.
        :param w3: Third word.
        :return: Probability of w3 given w1 and w2.
        """
        vocab_size = len(self.known_words)

        # Get the bigram count for (w1, w2)
        bigram_count = self.bigram_counts[w1][w2]

        if bigram_count > 0:
            return (self.trigram_counts[(w1, w2)][w3] + self.smoothing_coef) / (bigram_count + self.smoothing_coef * vocab_size)
        else:
            # Handle the case where (w1, w2) is unknown, fallback to a smoothed probability for OOV
            return (self.trigram_counts[('OOV', 'OOV')][w3] + self.smoothing_coef) / (self.bigram_counts['OOV']['OOV'] + self.smoothing_coef * vocab_size)

    def replace_unknowns(self, sentence):
        """
        Replace unknown words in the sentence with 'OOV'.
        :param sentence: A list of words (tokens).
        :return: List of words with unknown words replaced by 'OOV'.
        """
        return ['OOV' if word not in self.known_words else word for word in sentence]

    def sentence_perplexity(self, sentence):
        """
        Calculate the perplexity of a sentence.
        :param sentence: A sentence string.
        :return: The perplexity of the sentence.
        """
        tokens = self.tokenizer.tokenize(sentence.lower())
        tokens = self.replace_unknowns(tokens)

        T = len(tokens)

        if T == 0:
            return float('inf')

        # Initial unigram and bigram probabilities for first two tokens
        prob = math.pow(self.unigram_probability(tokens[0]), 1/T)
        if T > 1:
            prob *= math.pow(self.bigram_probability(tokens[0], tokens[1]), 1/T)

        # Trigram probabilities for the rest of the tokens
        for i in range(2, T):
            prob *= math.pow(self.trigram_probability(tokens[i - 2], tokens[i - 1], tokens[i]), 1/T)

        return 1 / prob if prob > 0 else float('inf')


In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
# Load the XLM-Roberta tokenizer
tokenizer_multilingual = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
def evaluate_model(model, validation_sentences):
    """
    Evaluate the model by calculating the perplexity on validation sentences.
    :param model: The TrigramModel instance to evaluate.
    :param validation_sentences: List of validation sentences.
    :return: Average perplexity of the model on the validation set.
    """
    total_perplexity = 0
    count = 0

    for sentence in validation_sentences:
        perplexity = model.sentence_perplexity(sentence)
        total_perplexity += perplexity
        count += 1

    return total_perplexity / count if count > 0 else float('inf')

smoothing_coef = 0.1

Finnish_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Finnish_model.train(train_df[train_df['lang'] == 'fi']['question'])

Japanese_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Japanese_model.train(train_df[train_df['lang'] == 'ja']['question'])

Russian_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Russian_model.train(train_df[train_df['lang'] == 'ru']['question'])

Context_model = TrigramModel(tokenizer_multilingual, smoothing_coef)
Context_model.train(train_df['context'])

validation_fi = val_df[val_df['lang'] == 'fi']['question']
validation_ja = val_df[val_df['lang'] == 'ja']['question']
validation_ru = val_df[val_df['lang'] == 'ru']['question']
validation_context = val_df['context']


# Evaluate models
finnish_perplexity = evaluate_model(Finnish_model, validation_fi)
print(f"Finnish Model Perplexity: {finnish_perplexity:.2f}")

japanese_perplexity = evaluate_model(Japanese_model, validation_ja)
print(f"Japanese Model Perplexity: {japanese_perplexity:.2f}")

russian_perplexity = evaluate_model(Russian_model, validation_ru)
print(f"Russian Model Perplexity: {russian_perplexity:.2f}")

context_perplexity = evaluate_model(Context_model, validation_context)
print(f"Context Model Perplexity: {context_perplexity:.2f}")


 ####  2. Bigram model (Tom)

The dataset for each langauge is rather small. In order to avoid overfitting I chose to implement a simple model - the bigram one.

In [ ]:
from collections import defaultdict, Counter
import math

class BigramModel:
    def __init__(self, tokenizer, smoothing_coef):
        # Initialize the bigram and unigram counts
        self.bigram_counts = defaultdict(Counter)
        self.unigram_counts = Counter()
        self.known_words = set()
        self.tokenizer = tokenizer
        self.smoothing_coef = smoothing_coef

    def train(self, sentences):
        """
        Train the bigram model on a list of tokenized sentences.
        :param sentences: List of sentences where each sentence is a list of words.
        """
        tokenized_sentences = [self.tokenizer.tokenize(sentence.lower()) for sentence in sentences]

        for sentence in tokenized_sentences:
            # Replace first occurrences with 'OOV' and update known_words set
            for i in range(len(sentence)):
                word = sentence[i]
                if word not in self.known_words:
                    sentence[i] = 'OOV'
                    self.known_words.add(word)

            # Count bigrams and unigrams
            for i in range(len(sentence) - 1):
                self.unigram_counts[sentence[i]] += 1
                self.bigram_counts[sentence[i]][sentence[i + 1]] += 1
            self.unigram_counts[sentence[-1]] += 1  # Count the last word as a unigram

    def unigram_probability(self, word):
        """
        Calculate the unigram probability P(word).
        :param word: The word for which to calculate the probability.
        :return: The probability of the word.
        """
        total_count = sum(self.unigram_counts.values())
        vocab_size = len(self.known_words)  # Unique words
        if word in self.unigram_counts:
            return self.unigram_counts[word] / total_count
        else:
            return self.unigram_counts['OOV'] / total_count if 'OOV' in self.unigram_counts else 0

    def bigram_probability(self, w1, w2):
        """
        Calculate the bigram probability P(w2 | w1).
        :param w1: Previous word.
        :param w2: Current word.
        :return: Probability of w2 given w1.
        """
        vocab_size = len(self.known_words)  # Unique words
        if self.unigram_counts[w1] > 0:
            return (self.bigram_counts[w1][w2] + self.smoothing_coef) / (self.unigram_counts[w1] + self.smoothing_coef * vocab_size)
        else:
            return (self.bigram_counts['OOV'][w2] + self.smoothing_coef) / (self.unigram_counts['OOV'] + self.smoothing_coef * vocab_size)

    def replace_unknowns(self, sentence):
        """
        Replace unknown words in the sentence with 'OOV'.
        :param sentence: A list of words (tokens).
        :return: List of words with unknown words replaced by 'OOV'.
        """
        return ['OOV' if word not in self.known_words else word for word in sentence]

    def sentence_preplexity(self, sentence):
        """
        Calculate the perplexity of a sentence.
        :param sentence: A sentence string.
        :return: The perplexity of the sentence.
        """
        tokens = self.tokenizer.tokenize(sentence.lower())
        tokens = self.replace_unknowns(tokens)

        T = len(tokens)

        if T == 0:
          return float('inf')

        prob = math.pow(self.unigram_probability(tokens[0]), 1/T)
        for i in range(T - 1):
          prob *= math.pow(self.bigram_probability(tokens[i], tokens[i + 1]), 1/T)

        if prob > 0:
          return 1 / prob  # Perplexity formula
        else:
          #print unigram probabilty for first token
          return float('inf')
        return prob

Now that we have a working model template, we can apply it to the Russian / Japanese / Finnish questions and the English context.

In [ ]:
!pip install transformers

In [ ]:
from transformers import AutoTokenizer
# Load the XLM-Roberta tokenizer
tokenizer_multilingual = AutoTokenizer.from_pretrained("xlm-roberta-base")

In [ ]:
def evaluate_model(model, validation_sentences):
    """
    Evaluate the model by calculating the perplexity on validation sentences.
    :param model: The BigramModel instance to evaluate.
    :param validation_sentences: List of validation sentences.
    :return: Average perplexity of the model on the validation set.
    """
    total_perplexity = 0
    count = 0

    for sentence in validation_sentences:
        perplexity = model.sentence_preplexity(sentence)
        total_perplexity += perplexity
        count += 1

    return total_perplexity / count if count > 0 else float('inf')

smoothing_coef = 0.1 # Random choice. Not possible to calibrate because we're using the validation set as a testing set.

Finnish_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Finnish_model.train(train_df[train_df['lang'] == 'fi']['question'])

Japanese_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Japanese_model.train(train_df[train_df['lang'] == 'ja']['question'])

Russian_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Russian_model.train(train_df[train_df['lang'] == 'ru']['question'])

Context_model = BigramModel(tokenizer_multilingual, smoothing_coef)
Context_model.train(train_df['context'])

validation_fi = val_df[val_df['lang'] == 'fi']['question']
validation_ja = val_df[val_df['lang'] == 'ja']['question']
validation_ru = val_df[val_df['lang'] == 'ru']['question']
validation_context = val_df['context']


# Evaluate models
finnish_perplexity = evaluate_model(Finnish_model, validation_fi)
print(f"Finnish Model Perplexity: {finnish_perplexity:.2f}")

japanese_perplexity = evaluate_model(Japanese_model, validation_ja)
print(f"Japanese Model Perplexity: {japanese_perplexity:.2f}")

russian_perplexity = evaluate_model(Russian_model, validation_ru)
print(f"Russian Model Perplexity: {russian_perplexity:.2f}")

context_perplexity = evaluate_model(Context_model, validation_context)
print(f"Context Model Perplexity: {context_perplexity:.2f}")


To check: Why the perplexity of Trigram is higher than the bigram? Shouldn't it be the opposite?... (sofia)

# Week 38


Let k be the number of members in your group. For each of the three languages
Finnish, Japanese and Russian separately, using the training data, train k different classifiers that receive the document (context) and question as input
and predict whether the question is answerable or impossible given the context.

Evaluate the classifiers on the respective validation sets, report and analyse the performance for each language and compare the scores across languages.
The classifiers can use machine translation, linguistic/lexical features (e.g.,
bag-of-words, n-gram counts, word overlap) word embeddings, or word/sentence
representations from (multilingual) neural language models.8 You can also train
or fine-tune your own neural language models on the dataset. Different from
1(c), however, they must be learned rather than rule-based. Motivate your
choice of features and classifier

#### 1. Neural Network with mean embeddings (Tom)

In [ ]:
import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

In [ ]:
!pip install transformers torch
!pip install tqdm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer, DistilBertModel
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.metrics import confusion_matrix
import random

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')
distilbert_model = DistilBertModel.from_pretrained('distilbert-base-multilingual-cased')

def get_mean_pooled_embedding(sentences):
    inputs = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        outputs = distilbert_model(**inputs)
    last_hidden_states = outputs.last_hidden_state

    # Mean pooling over the token embeddings
    mean_pooled = last_hidden_states.mean(dim=1)
    return mean_pooled

In [ ]:
import os
import numpy as np
import pandas as pd  # Make sure to import pandas
from google.colab import drive
from tqdm import tqdm  # Import tqdm for progress bar

def ensure_directory_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def pre_process(data, use_embeddings=False, save_embeddings=False, filename=None):
    # Ensure the directory exists if either use_embeddings or save_embeddings is True
    if use_embeddings or save_embeddings:
        # Mount Google Drive
        drive.mount('/content/drive')
        ensure_directory_exists(os.path.dirname(filename))

    # Check if embeddings file already exists and if we want to use it
    if use_embeddings and os.path.exists(filename):
        print("Loading existing embeddings...")
        return load_embeddings(filename)

    # If file doesn't exist or we don't want to use it, process data
    context_embeddings = []
    question_embeddings = []

    for index, row in tqdm(data.iterrows(), total=len(data), desc="Processing data"):
        context = row['context']
        question = row['question']
        context_embedding = get_mean_pooled_embedding(context)
        question_embedding = get_mean_pooled_embedding(question)
        context_embeddings.append(context_embedding)
        question_embeddings.append(question_embedding)

    context_tensor = tf.squeeze(torch.stack(context_embeddings), axis=1)
    question_tensor = tf.squeeze(torch.stack(question_embeddings), axis=1)
    label_tensor = tf.convert_to_tensor(data['answerable'].astype(int), dtype=tf.int32)

    # Save embeddings to Google Drive if requested
    if save_embeddings and filename:
        np.savez(filename, context=context_tensor.numpy(), question=question_tensor.numpy(), labels=label_tensor.numpy())
        print(f"Embeddings saved to {filename}")

    return context_tensor, question_tensor, label_tensor

def load_embeddings(filename):
    data = np.load(filename)
    context_tensor = tf.convert_to_tensor(data['context'])
    question_tensor = tf.convert_to_tensor(data['question'])
    label_tensor = tf.convert_to_tensor(data['labels'], dtype=tf.int32)
    return context_tensor, question_tensor, label_tensor

In [ ]:
def set_seed(seed_value=42):
    # Setting the seed for Python's random module
    random.seed(seed_value)

    # Setting the seed for NumPy
    np.random.seed(seed_value)

    # Setting the seed for TensorFlow
    tf.random.set_seed(seed_value)

    # Ensuring TensorFlow works deterministically (optional, but useful for reproducibility)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    os.environ['TF_DETERMINISTIC_OPS'] = '1'

def create_and_train_model(training_data, use_embeddings=False, save_embeddings=False, filename=None):
    # Process Data
    context_tensor, question_tensor, label_tensor = pre_process(training_data, use_embeddings, save_embeddings, filename)

    # Create model
    input_1 = layers.Input(shape=(context_tensor.shape[1],))
    input_2 = layers.Input(shape=(question_tensor.shape[1],))

    # Concatenate the two input vectors
    concatenated = layers.Concatenate()([input_1, input_2])

    # Hidden layer
    hidden_layer = layers.Dense(128, activation='relu')(concatenated)

    # Output layer (binary classification, use a single unit with sigmoid activation)
    output = layers.Dense(1, activation='sigmoid')(hidden_layer)

    # Define the model
    model = models.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model (using binary crossentropy loss for binary classification)
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model
    model.fit([context_tensor, question_tensor], label_tensor, epochs=10, batch_size=32)
    return model

def evaluate_model(model, validation_data, use_embeddings=False, save_embeddings=False, filename=None):
    # Process Data
    context_tensor, question_tensor, label_tensor = pre_process(validation_data, use_embeddings, save_embeddings, filename)

    # Evaluate the model
    threshold = 0.5
    loss, accuracy = model.evaluate([context_tensor, question_tensor], label_tensor)

    # Calculate Confusion Matrix
    predictions_prob = model.predict([context_tensor, question_tensor])
    predictions = (predictions_prob >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(label_tensor, predictions).ravel()

    # Calculate TPR and FPR
    tpr = tp / (tp + fn) if (tp + fn) > 0 else 0.0  # True Positive Rate
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0.0  # False Positive Rate

    return loss, accuracy, tpr, fpr

In [ ]:
set_seed()
base_dir = '/content/drive/My Drive/nlp/week3'
use_embeddings = True
save_embeddings = True

results = {}

for lang in ['ru', 'ja', 'fi']:
    # Filter training and validation data for the specific language
    train_data = train_df[train_df['lang'] == lang]
    val_data = val_df[val_df['lang'] == lang]

    # Create and train the model
    print(f"Training model for language: {lang}")
    train_file_name = os.path.join(base_dir, f"{lang}_embeddings.npz")
    model = create_and_train_model(train_data, use_embeddings, save_embeddings, train_file_name)

    # Evaluate the model
    print(f"Evaluating model for language: {lang}")
    test_file_name = os.path.join(base_dir, f"{lang}_test_embeddings.npz")
    loss, accuracy, tpr, fpr = evaluate_model(model, val_data, use_embeddings, save_embeddings, test_file_name)

    # Store the results
    results[lang] = {
        'Loss': loss,
        'Accuracy': accuracy,
        'True Positive Rate (TPR)': tpr,
        'False Positive Rate (FPR)': fpr
    }

# Print the results for each language model
for lang, metrics in results.items():
    print(f"Results for {lang}:")
    print(f"  Loss: {metrics['Loss']:.4f}")
    print(f"  Accuracy: {metrics['Accuracy']:.4f}")
    print(f"  True Positive Rate (TPR): {metrics['True Positive Rate (TPR)']:.4f}")
    print(f"  False Positive Rate (FPR): {metrics['False Positive Rate (FPR)']:.4f}")
    print()  # Just for better readability

#### 2. Logistic Regression with Bag-of-Words (Sofia)

In [ ]:
import nltk
import pandas as pd
import warnings
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Fix the "copy warning" for readable output
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Import Datasets
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Split questions by the language
finnish_questions = df[df['lang'] == 'fi']
japanese_questions = df[df['lang'] == 'ja']
russian_questions = df[df['lang'] == 'ru']

languages = ['fi', 'ru']

# Load stop words into lists
nltk.download('stopwords')
stop_words_english = list(stopwords.words('english'))  # Преобразование к списку
stop_words_finnish = list(stopwords.words('finnish'))
stop_words_russian = list(stopwords.words('russian'))

for lang in languages:
    questions = df[df['lang'] == lang]

    # Generating stopword list for each case
    if lang == 'fi':
        stop_words = stop_words_finnish + stop_words_english
    elif lang == 'ru':
        stop_words = stop_words_russian + stop_words_english
    else:
        stop_words = stop_words_english

    # Combine context and question into a single input feature
    questions.loc[:, 'combined_input'] = questions.loc[:, 'context'] + ' ' + questions.loc[:, 'question']

    # Create a bag-of-words representation with removal of stop-words
    vectorizer = CountVectorizer(stop_words=stop_words)
    X = vectorizer.fit_transform(questions['combined_input'])
    y = questions['answerable'].astype(int)  # Convert 'True'/'False' to 0/1

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # Train a logistic regression model
    model = LogisticRegression(max_iter=1000)
    model.fit(X_train, y_train)

    # Make predictions on the test set
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    print(f"Accuracy for language {lang}: {accuracy:.2f}")
    print(classification_report(y_test, y_pred))


Проблема может быть в том, что чтобы понять - можно ли ответить на вопрос или нет - нужно смотреть на реальные ключевые слова и упускать слова без смысла

The model has excellent performance for class 1 (answerable), with high precision, recall, and F1-score.
However, it struggles with class 0 (impossible) questions, as indicated by its low precision, recall, and F1-score.
Accuracy is misleading in this case due to the class imbalance. Although 92% of the predictions are correct, this is mainly due to the overwhelming presence of class 1.

To improve performance on class 0:
Use class weighting to penalize the model for misclassifying class 0.
Resample the data to balance the classes (e.g., oversample class 0 or undersample class 1).
Experiment with more sophisticated models or feature engineering techniques.

In [ ]:
# prompt: count questions in dataset with 0 (not answerable)

not_answerable_count = train_df[train_df['answerable'] == False].shape[0]
print(f"Number of questions not answerable: {not_answerable_count}")

answerable_count = train_df[train_df['answerable'] == True].shape[0]
print(f"Number of questions answerable: {answerable_count}")


Japonese text requires specific tokenizer:

In [ ]:
# Load tokenizer directly
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer_ja = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
#model_ja = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

def tokenize_japanese_sentences(sentence):
    tokens = tokenizer_ja.tokenize(sentence)
    return " ".join(tokens)

japanese_questions = df[df['lang'] == 'ja']['question']

# Apply tokenization to the 'japanese_questions' series
tokenized_japanese_questions = japanese_questions.apply(tokenize_japanese_sentences)


# Print the original and tokenized questions
#for original, tokenized in zip(japanese_questions.head(5), tokenized_japanese_questions.head(5)):
#    print(f"Original: {original}")
#    print(f"Tokenized: {tokenized}")
#    print("-" * 50)

In [ ]:
japanese_questions.shape

In [ ]:
japanese_questions.head(3)

In [ ]:
tokenized_japanese_questions.shape

In [ ]:
tokenized_japanese_questions.head(3)

In [ ]:
# Now apply all the same to the Japonese language - tokenized japonesse text
import nltk
import pandas as pd
import warnings
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

# Fix the "copy warning" for readable output
warnings.filterwarnings("ignore", category=pd.errors.SettingWithCopyWarning)

# Import Datasets
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Split questions by the language

japanese_questions = df[df['lang'] == 'ja']
lang = 'ja'
#languages = ['fi', 'ru']

# Load stop words into lists
# nltk.download('stopwords')
stop_words_english = list(stopwords.words('english'))  # Преобразование к списку

# Generate stopword list for Finnish
stop_words = stop_words_english
questions = japanese_questions

# Combine context and question into a single input feature
questions.loc[:, 'combined_input'] = japanese_questions.loc[:, 'context'] + ' ' + tokenized_japanese_questions #.loc[:, 'question']

# Create a bag-of-words representation with removal of stop-words
vectorizer = CountVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(questions['combined_input'])
y = questions['answerable'].astype(int)  # Convert 'True'/'False' to 0/1

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy for language {lang}: {accuracy:.2f}")
print(classification_report(y_test, y_pred))


In [ ]:
"""
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

# Import Datasets
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Download the stopwords
nltk.download('stopwords')

# List of stop words
stop_words_en = stopwords.words('english')
stop_words_fi = list(stopwords.words('finnish'))  # Convert set to list
stop_words_ru = list(stopwords.words('russian'))  # Convert set to list

# List of languages to evaluate
languages = ['ja', 'ru', 'fi']

# Load tokenizers and models for Japanese language
tokenizer_ja = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ja-en")
model_ja = AutoModel.from_pretrained("Helsinki-NLP/opus-mt-ja-en")

def extract_embeddings_ja(train_lang_df, valid_lang_df):
    # Tokenize and extract embeddings for Japanese
    def get_embeddings(df):
        inputs = tokenizer_ja(df['combined_input'].tolist(), padding=True, truncation=True, return_tensors="pt")
        with torch.no_grad():
            embeddings = model_ja(**inputs).last_hidden_state.mean(dim=1)  # Use mean pooling
        return embeddings

    return get_embeddings(train_lang_df), get_embeddings(valid_lang_df)

for lang in languages:
    # Filter the training and validation data for the specific language
    train_lang_df = train_df[train_df['lang'] == lang]
    valid_lang_df = val_df[val_df['lang'] == lang]

    if len(train_lang_df) > 0:
        # Combine context and question into a single input feature
        train_lang_df['combined_input'] = train_lang_df['context'] + ' ' + train_lang_df['question']
        valid_lang_df['combined_input'] = valid_lang_df['context'] + ' ' + valid_lang_df['question']

        if lang == 'ja':
            # Extract embeddings for Japanese
            X_train, X_valid = extract_embeddings_ja(train_lang_df, valid_lang_df)
        else:
            # Use CountVectorizer for Finnish and Russian
            vectorizer = CountVectorizer(stop_words=stop_words_en if lang == 'fi' else stop_words_ru)
            X_train = vectorizer.fit_transform(train_lang_df['combined_input']).toarray()
            X_valid = vectorizer.transform(valid_lang_df['combined_input']).toarray()

        y_train = train_lang_df['answerable'].astype(int)
        y_valid = valid_lang_df['answerable'].astype(int)

        # Train a logistic regression model
        classifier = LogisticRegression(max_iter=1000)
        classifier.fit(X_train, y_train)

        # Make predictions on the validation set
        y_pred = classifier.predict(X_valid)

        # Evaluate the model
        accuracy = accuracy_score(y_valid, y_pred)
        print(f"Language: {lang}, Accuracy: {accuracy:.2f}")
        print(classification_report(y_valid, y_pred))
    else:
        print(f"No training data for language: {lang}")

  """


# Week 39

We now move from binary classification to span-based QA, i.e. identifying the
span in the document that answers the question.
Let k be the number of members in your group. Using the training data in
Finnish, Japanese and Russian separately, train k different sequence labellers,
which predict the tokens in a document context that constitute the answer to
the corresponding question. You can decide whether to train one model per
language or a single model for all three languages. Evaluate using a sequence labelling metric on the validation set, report and analyse the performance for each
language and compare the scores across languages. Note that if the question is
unanswerable, a correct output must be empty (contain no tokens).

#### (Tom)

In [ ]:
!pip install datasets
!pip install --upgrade torch accelerate

import pandas as pd

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 106.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 51.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# Import necessary libraries
import torch
from transformers import (
    DistilBertForQuestionAnswering,
    DistilBertTokenizerFast,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
import numpy as np

# Check and set the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load pre-trained model and tokenizer
model_name = 'distilbert-base-multilingual-cased'
tokenizer = DistilBertTokenizerFast.from_pretrained(model_name)
model = DistilBertForQuestionAnswering.from_pretrained(model_name).to(device)

# Function to prepare features for the model
def prepare_features(examples):
    # Tokenize the inputs with the tokenizer
    tokenized_examples = tokenizer(
        examples['question'],
        examples['context'],
        truncation=True,
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,  # Needed to map tokens to original text
        padding='max_length',
    )

    # Since one example might give rise to multiple features (due to truncation),
    # we need to keep track of the mapping between features and examples
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    offset_mapping = tokenized_examples["offset_mapping"]

    # Initialize lists to store the start and end positions
    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        # Get the input IDs and CLS index
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Get the sequence IDs to differentiate question and context tokens
        sequence_ids = tokenized_examples.sequence_ids(i)

        # Map the feature to its corresponding example
        sample_index = sample_mapping[i]
        answer = examples["answer"][sample_index]
        answer_start = examples["answer_start"][sample_index]
        context = examples["context"][sample_index]

        # If there is no answer, set the start and end positions to the CLS index
        if answer_start is None or answer_start == -1:
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            # Calculate the start and end character positions of the answer in the context
            start_char = answer_start
            end_char = answer_start + len(answer)

            # Find the start and end token indices in the feature
            token_start_index = 0
            while sequence_ids[token_start_index] != 1:
                token_start_index += 1

            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != 1:
                token_end_index -= 1

            # If the answer is out of the span (due to truncation), label it as CLS index
            if not (offsets[token_start_index][0] <= start_char and offsets[token_end_index][1] >= end_char):
                start_positions.append(cls_index)
                end_positions.append(cls_index)
            else:
                # Otherwise, find the start and end token indices that correspond to the answer
                # Note that the answer could be in the middle of the tokens due to tokenization
                # Adjust the start and end positions accordingly
                start_position = token_start_index
                end_position = token_end_index

                for idx in range(token_start_index, token_end_index + 1):
                    if offsets[idx][0] <= start_char and offsets[idx][1] > start_char:
                        start_position = idx
                    if offsets[idx][0] < end_char and offsets[idx][1] >= end_char:
                        end_position = idx
                        break

                start_positions.append(start_position)
                end_positions.append(end_position)

    # Add the start and end positions to the tokenized examples
    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions
    tokenized_examples.pop("offset_mapping")  # Remove offset_mapping as it's no longer needed

    return tokenized_examples

Using device: cuda


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Convert pandas DataFrames to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df[(train_df['lang'] == 'ru') | (train_df['lang'] == 'ja') | (train_df['lang'] == 'fi')])

# Apply the prepare_features function to the dataset
tokenized_train_dataset = train_dataset.map(
    prepare_features,
    batched=True,
    remove_columns=train_dataset.column_names,
)

# Set training arguments
training_args = TrainingArguments(
    output_dir='./results',
    eval_strategy='no',  # Disable evaluation during training
    save_strategy='no',     # You can choose when to save checkpoints
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=100,
    save_total_limit=2,
    fp16=True,
    dataloader_num_workers=2,
    report_to="none"  # This disables reporting to W&B
)

# Initialize the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    tokenizer=tokenizer,
    # compute_metrics can be omitted since we're not evaluating
)


# Start training
trainer.train()

# Save the trained model
#trainer.save_model('./trained_model')

Map:   0%|          | 0/6410 [00:00<?, ? examples/s]

Step,Training Loss
100,3.526700
200,2.744200
300,2.763500
400,2.610300
500,2.211100
600,2.035600
700,1.937500
800,1.957400
900,1.679400
1000,1.547500


TrainOutput(global_step=1236, training_loss=2.149041206705532, metrics={'train_runtime': 305.3471, 'train_samples_per_second': 64.746, 'train_steps_per_second': 4.048, 'total_flos': 1937258840724480.0, 'train_loss': 2.149041206705532, 'epoch': 3.0})

In [ ]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00


In [ ]:
# Import necessary libraries
import numpy as np
from datasets import Dataset
import evaluate
from collections import defaultdict

# Load the evaluation metric
metric = evaluate.load("squad")

val_dataset = Dataset.from_pandas(val_df[(val_df['lang'] == 'ru') | (val_df['lang'] == 'ja') | (val_df['lang'] == 'fi')])
val_dataset = val_dataset.map(lambda example, idx: {'id': str(idx)}, with_indices=True)

# Prepare the validation features
def prepare_validation_features(examples):
    tokenized_examples = tokenizer(
        examples['question'], examples['context'],
        truncation=True, max_length=384, stride=128,
        return_overflowing_tokens=True, return_offsets_mapping=True,
        padding='max_length'
    )
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    tokenized_examples["example_id"] = [examples["id"][sample_mapping[i]] for i in range(len(tokenized_examples["input_ids"]))]
    return tokenized_examples

# Tokenize the validation dataset
tokenized_val_dataset = val_dataset.map(prepare_validation_features, batched=True, remove_columns=val_dataset.column_names)

# Make predictions on the validation dataset
predictions = trainer.predict(tokenized_val_dataset)

# Post-process predictions to get the final answers
def postprocess_predictions(examples, features, predictions):
    all_start_logits, all_end_logits = predictions
    example_id_to_index = {str(k): i for i, k in enumerate(examples["id"])}
    features_per_example = defaultdict(list)

    for i, feature in enumerate(features):
        features_per_example[str(feature["example_id"])].append(i)

    final_predictions = {}
    for example_id, feature_indices in features_per_example.items():
        context = examples[example_id_to_index[example_id]]["context"]
        best_answer, max_score = "", -float('inf')

        for feature_index in feature_indices:
            start_logits, end_logits, offsets = all_start_logits[feature_index], all_end_logits[feature_index], features[feature_index]["offset_mapping"]
            start_idx, end_idx = np.argmax(start_logits), np.argmax(end_logits)

            if start_idx <= end_idx and start_idx < len(offsets) and end_idx < len(offsets):
                start_char, end_char = offsets[start_idx][0], offsets[end_idx][1]
                predicted_answer = context[start_char:end_char]
                score = start_logits[start_idx] + end_logits[end_idx]

                if score > max_score:
                    best_answer, max_score = predicted_answer, score

        final_predictions[example_id] = best_answer if best_answer != "" else " "

    return final_predictions

# Generate final predictions
final_predictions = postprocess_predictions(val_dataset, tokenized_val_dataset, predictions.predictions)

# Convert 'val_dataset' to a pandas DataFrame and filter by language
val_df_with_ids = val_dataset.to_pandas().astype({'id': str})
val_df_with_ids.loc[val_df_with_ids['answer_start'] == -1, 'answer'] = " "
languages = ['ru', 'fi', 'ja']
val_dfs_by_language = {lang: val_df_with_ids[val_df_with_ids['lang'] == lang].reset_index(drop=True) for lang in languages}
val_dfs_by_language['all'] = val_df_with_ids[val_df_with_ids['lang'].isin(languages)].reset_index(drop=True)

# Evaluate and compute metrics per language
for lang, lang_df in val_dfs_by_language.items():
    if lang_df.empty:
        print(f"No examples found for language: {lang}\n")
        continue

    lang_dataset = Dataset.from_pandas(lang_df)
    references = [{"id": ex["id"], "answers": {"text": [ex["answer"]], "answer_start": [ex["answer_start"]]}} for ex in lang_dataset]
    lang_ids = set(lang_df['id'])
    predictions_formatted = [{"id": k, "prediction_text": v} for k, v in final_predictions.items() if k in lang_ids]

    if len(predictions_formatted) != len(references):
        print(f"Mismatch in predictions/references for language {lang}\n")
        continue

    # Compute and print metrics
    results = metric.compute(predictions=predictions_formatted, references=references)
    print(f"Language: {lang}\nExact Match: {results['exact_match']:.2f}\nF1 Score: {results['f1']:.2f}\n")

Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

Language: ru
Exact Match: 41.16
F1 Score: 27.95

Language: fi
Exact Match: 44.13
F1 Score: 28.41

Language: ja
Exact Match: 50.00
F1 Score: 26.79

Language: all
Exact Match: 45.22
F1 Score: 27.75



In [ ]:
!pip install googletrans==4.0.0-rc1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.6 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-4.0.0rc1-py3-none-any.whl size=17397 sha256=c96c5f345df8912d7b04761b7536a24b570ca99d4084966f2a02dec802e0b6dd
  Stored in directory: /root/.cache/pip/wheels/c0/59/9f/7372f0cf70160fe61b528532e1a7c8498c4becd6bcffb022de
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:

In [ ]:
from googletrans import Translator
translator = Translator()

# Convert 'val_dataset' to a pandas DataFrame
val_df_with_ids = val_dataset.to_pandas()

# Ensure that 'id' is a string type to match the keys in 'final_predictions'
val_df_with_ids['id'] = val_df_with_ids['id'].astype(str)

# Convert 'val_dataset' to a pandas DataFrame
val_df_with_ids = val_dataset.to_pandas()
val_df_with_ids['id'] = val_df_with_ids['id'].astype(str)

# Initialize the translator
translator = Translator()

# Iterate over the first 10 examples
for idx in range(10, min(20, len(val_df_with_ids))):
    # Get the example
    example = val_df_with_ids.iloc[idx]
    example_id = example['id']
    question = example['question']
    context = example['context']
    actual_answer = example['answer']
    predicted_answer = final_predictions.get(example_id, "")
    language = example['lang']

    # Translate the question into English
    try:
        translation = translator.translate(question, dest='en')
        question_en = translation.text
    except Exception as e:
        question_en = "Translation unavailable"
        print(f"Error translating question {idx+1}: {e}")

    print(f"Example {idx+1}:")
    print(f"Question ({language}): {question}")
    print(f"Translated Question: {question_en}")
    print(f"Context: {context}\n")
    print(f"Actual Answer: {actual_answer}")
    print(f"Actual Answer_Start: {example['answer_start']}")
    print(f"Predicted Answer: {predicted_answer}")
    print("-" * 80)

Example 11:
Question (fi): Mikä on Jaco Pastoriuksen tunnetuin kappale?
Translated Question: What is Jaco Pastorius' most famous song?
Context: Birdland marked the peak of Weather Report's commercial career with the release of "Heavy Weather". With the addition of Jaco Pastorius, the band was able to push its music to the "height of its popularity", and with that came "Birdland." "Birdland" served as a tribute to the famous New York City jazz club that hosted many famous jazz musicians, which operated on Broadway from 1949 through 1965. This was the club, which he frequented almost daily, where Zawinul heard Count Basie, Louis Armstrong, Duke Ellington, and Miles Davis. It was also where he met his wife, Maxine. Looking back, Zawinul claimed, "The old Birdland was the most important place in my life." The song was also named in honor of the man after whom the club was named, Charlie Parker, the 'Bird' himself. 

Actual Answer: Birdland
Actual Answer_Start: 0
Predicted Answer:  
-------

#### 2. Sofia

In [ ]:
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizerFast, BertForTokenClassification, Trainer, TrainingArguments
import numpy as np

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

train_data = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
valid_data = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

# Load data
train_data = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"], columns=['context', 'question', 'answerable', 'answer', 'lang'], filters=[('lang', 'in', ['ja','fi','ru'])])
valid_data = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"], columns=['context', 'question', 'answerable', 'answer', 'lang'], filters=[('lang', 'in', ['ja','fi','ru'])])


# Define Dataset class
class ContextAnswerDataset(Dataset):
    def __init__(self, data, tokenizer, max_length=256):  # Reduced max_length
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        context = self.data.iloc[idx]['context']
        answer = self.data.iloc[idx]['answer']

        # Tokenize context and answer
        encoding = self.tokenizer(context, padding='max_length', truncation=True, max_length=self.max_length, return_tensors="pt")
        labels = self.create_labels(context, answer, encoding)

        encoding = {key: val.squeeze() for key, val in encoding.items()}  # remove batch dimension
        encoding['labels'] = torch.tensor(labels, dtype=torch.long)

        return encoding

    def create_labels(self, context, answer, encoding):
        """ Create labels where tokens are classified as 1 if part of the answer, else 0 """
        labels = np.zeros(len(encoding['input_ids'][0]), dtype=int)  # 0 = not part of answer, 1 = part of answer

        # Tokenize answer and search for it within the context
        answer_tokens = self.tokenizer.tokenize(answer)
        answer_ids = self.tokenizer.convert_tokens_to_ids(answer_tokens)
        context_ids = encoding['input_ids'][0].tolist()

        for i in range(len(context_ids) - len(answer_ids) + 1):
            if context_ids[i:i+len(answer_ids)] == answer_ids:
                labels[i:i+len(answer_ids)] = 1

        return labels

# Tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-multilingual-cased')

# Create datasets
train_dataset = ContextAnswerDataset(train_data, tokenizer)
valid_dataset = ContextAnswerDataset(valid_data, tokenizer)

# Model: BertForTokenClassification
model = BertForTokenClassification.from_pretrained('bert-base-multilingual-cased', num_labels=2)

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,  # Reduced number of epochs
    per_device_train_batch_size=4,  # Reduced batch size
    per_device_eval_batch_size=4,  # Reduced batch size
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch",
    fp16=True,  # Enable mixed precision training if using a compatible GPU
    gradient_accumulation_steps=2,  # Accumulate gradients for larger effective batch size
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=valid_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Save the model
model.save_pretrained("./saved_model")
tokenizer.save_pretrained("./saved_model")


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarni

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
!pip install Levenshtein

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.9 MB/s eta 0:00:00


In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import BertTokenizer, BertForTokenClassification
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
import Levenshtein as lev  # Import the Levenshtein library

# Load the tokenizer and model from the checkpoint directory
model_save_path = "./saved_model"
tokenizer = BertTokenizer.from_pretrained(model_save_path)
model = BertForTokenClassification.from_pretrained(model_save_path)
model.eval()  # Set model to evaluation mode

# Set the maximum sequence length
MAX_LENGTH = 512

valid_data = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"], columns=['context', 'question', 'answerable', 'answer', 'lang', 'answer_start'], filters=[('lang', 'in', ['ja'])])

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Store overall true and predicted labels
all_true_labels = []
all_predicted_labels = []

# Variables to track proximity accuracy and Levenshtein count
correct_count = 0
lev_distance_count = 0
total_count = 0

for context, question, answerable, real_answer, lang, answer_start in list(valid_data.itertuples(index=False, name=None)):
    # Tokenize the context and question
    inputs = tokenizer(context, question, truncation=True, padding='max_length', max_length=MAX_LENGTH, return_tensors="pt")

    # Move the input tensors to the GPU if available
    inputs = {key: val.to(device) for key, val in inputs.items()}

    # Pass the inputs through the model
    with torch.no_grad():  # Disable gradient calculation during inference
        outputs = model(**inputs)

    # The logits represent predictions (before applying activation function like softmax)
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=-1).cpu().numpy()[0]  # Get predicted labels

    # Convert token IDs to tokens (words)
    input_ids = inputs['input_ids'].cpu().numpy()[0]  # Get input token ids
    tokens = tokenizer.convert_ids_to_tokens(input_ids)

    # Create true label array
    true_labels = np.zeros(len(tokens), dtype=int)
    true_labels[answer_start:answer_start + len(real_answer.split())] = 1  # Set true labels based on actual answer length

    # Collect all true and predicted labels for metric calculations
    all_true_labels.extend(true_labels)
    all_predicted_labels.extend(predictions)

    # Find the first contiguous set of predicted answer tokens in the context
    predicted_answer_tokens = []
    in_answer = False

    for i in range(len(predictions)):
        if predictions[i] == 1:  # If the token is part of the answer
            predicted_answer_tokens.append(tokens[i])
            in_answer = True
        elif in_answer:
            # Stop if we were in an answer sequence and hit a non-answer token
            break

    # Convert the list of predicted answer tokens to a string
    first_adjacent_answer = tokenizer.convert_tokens_to_string(predicted_answer_tokens)

    # Calculate Levenshtein Distance
    distance = lev.distance(first_adjacent_answer.strip(), real_answer.strip())

    # Define a dynamic threshold based on the length of the real answer
    answer_length = len(real_answer.strip())
    if answer_length <= 3:
        threshold = 1  # Short answers
    elif answer_length <= 10:
        threshold = 2  # Medium answers
    else:
        threshold = max(1, round(answer_length * 0.1))  # Long answers

    # Check if the distance is within the threshold
    if distance <= threshold:
        lev_distance_count += 1

    # Print for debugging
    if first_adjacent_answer != "":
        print(f"Predicted: '{first_adjacent_answer}' - Real: '{real_answer.strip()}' - Levenshtein Distance: {distance} - Threshold: {threshold}")

    # Count total predictions
    total_count += 1

# Convert lists to numpy arrays for metric calculations
all_true_labels = np.array(all_true_labels)
all_predicted_labels = np.array(all_predicted_labels)

# Calculate overall metrics
accuracy = accuracy_score(all_true_labels, all_predicted_labels)
precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_predicted_labels, average='binary')

# Print performance metrics
print(f"Accuracy (Token Classification): {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print(f"Levenshtein Distance Count (successful predictions): {lev_distance_count}/{total_count} = {lev_distance_count / total_count:.4f}")


Predicted: '##5' - Real: '−5 °C' - Levenshtein Distance: 5 - Threshold: 2
Predicted: '54' - Real: 'former wrestlers wishing to branch out (namely, those ranked below "yokozuna" or "ōzeki" must have spent at least 60 tournaments in the top "makuuchi" division or 25 in the titled "san'yaku" ranks)' - Levenshtein Distance: 196 - Threshold: 20
Predicted: '##sutomu Sato' - Real: 'Tsutomu Sato' - Levenshtein Distance: 2 - Threshold: 1
Predicted: '600' - Real: 'agriculture' - Levenshtein Distance: 11 - Threshold: 1
Predicted: '68' - Real: '177.0 km2' - Levenshtein Distance: 9 - Threshold: 2
Predicted: '1926' - Real: 'January 1926' - Levenshtein Distance: 8 - Threshold: 1
Predicted: 'David Keith' - Real: 'Takaaki Ishibashi' - Levenshtein Distance: 13 - Threshold: 2
Predicted: ', 362 square miles' - Real: '184,827 km2' - Levenshtein Distance: 17 - Threshold: 1
Predicted: 'Peter Rojas' - Real: 'Peter Rojas.' - Levenshtein Distance: 1 - Threshold: 1
Predicted: '15' - Real: '15' - Levenshtein Dist

# Week 40

We now introduce open QA, i.e. generating an answer to a question even when
it is not extracted as a span from a document.
While for all answerable questions in the dataset, the English answer is
available, for some of the questions in the dataset, the answer in the same
language as the question is also available, in the answer inlang field. Use this
subset of the questions in Finnish, Japanese and Russian to train (or fine-tune)
a model that receives the question and context as input and generates the in-
language answer.10 You can decide whether to train one model per language or
a single model for all three languages.
If your group contains at least two members, additionally train an encoder-
decoder model that receives only the question as input and generates the in-
language answer.
If your group contains at least three members, additionally train an encoder-
decoder model that receives only the English answer as input and generates the
in-language answer.
Evaluate using a text generation evaluation metric on the validation set,
compare the results across languages and models and discuss them

#### 1. Sofia

In [ ]:
!pip install transformers datasets

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.8 MB/s eta 0:00:00


In [ ]:
# Required Libraries
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainingArguments
from datasets import Dataset

In [ ]:
splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])

train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
finnish_questions = df[df['lang'] == 'fi']
japanese_questions = df[df['lang'] == 'ja']
russian_questions = df[df['lang'] == 'ru']

In [ ]:
# Concatenate the language-specific datasets for multi-language training
multi_lang_train_df = pd.concat([finnish_questions, japanese_questions, russian_questions])

# Convert Pandas DataFrame to Hugging Face Dataset
train_dataset = Dataset.from_pandas(multi_lang_train_df)
val_dataset = Dataset.from_pandas(val_df)

# Load pre-trained T5 model and tokenizer for text generation
model_name = "t5-base"  # You can choose another model (like 't5-small' or 't5-large')
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)

# Define a preprocessing function with handling for missing values
def preprocess_function(examples):
    # Ensure that both the question and context fields are strings and handle missing values
    inputs = [
        "question: " + (str(question) if question else "") + " context: " + (str(context) if context else "")
        for question, context in zip(examples['question'], examples['context'])
    ]
    targets = [(str(answer) if answer else "") for answer in examples['answer_inlang']]  # Handle missing targets

    # Tokenize inputs and targets
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding='max_length')
    labels = tokenizer(targets, max_length=128, truncation=True, padding='max_length')

    # Convert labels to tensors
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Apply the preprocessing function to datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)

# Set up training arguments
# Set a smaller batch size
training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,  # Reduce batch size to 4
    per_device_eval_batch_size=4,  # Reduce eval batch size
    num_train_epochs=3,
    weight_decay=0.01,
    predict_with_generate=True
)

# Define Trainer class for model fine-tuning
trainer = Trainer(
    model=model,                              # The pre-trained T5 model
    args=training_args,                       # Training arguments
    train_dataset=tokenized_train_dataset,    # Training dataset
    eval_dataset=tokenized_val_dataset,       # Evaluation dataset
    tokenizer=tokenizer,                      # Tokenizer
)

# Clear cache before training
torch.cuda.empty_cache()
# Start training
trainer.train()

# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

# Save the model after training
model.save_pretrained('./trained_t5_model')
tokenizer.save_pretrained('./trained_t5_model')


Results of the training attempt 1:

Epoch	Training Loss	Validation Loss

1	0.007900	0.035693

2	0.004500	0.032817

3	0.005300	0.031970

In [ ]:
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("t5-base")
print(model.forward.__code__.co_varnames)  # This will show the expected input variable names


('self', 'input_ids', 'attention_mask', 'decoder_input_ids', 'decoder_attention_mask', 'head_mask', 'decoder_head_mask', 'cross_attn_head_mask', 'encoder_outputs', 'past_key_values', 'inputs_embeds', 'decoder_inputs_embeds', 'labels', 'use_cache', 'output_attentions', 'output_hidden_states', 'return_dict', 'hidden_states', 'decoder_outputs', 'sequence_output', 'lm_logits', 'loss', 'loss_fct', 'output')


In [ ]:
print(tokenized_train_dataset.column_names)  # Check training dataset columns
print(tokenized_val_dataset.column_names)    # Check validation dataset columns


['question', 'context', 'lang', 'answerable', 'answer_start', 'answer', 'answer_inlang', '__index_level_0__', 'input_ids', 'attention_mask', 'labels']
['question', 'context', 'lang', 'answerable', 'answer_start', 'answer', 'answer_inlang', 'input_ids', 'attention_mask', 'labels']


In [ ]:
# Apply the model to the evaluation dataset and get predictions
predictions = trainer.predict(val_dataset)

# Get the generated tokens (for seq2seq tasks like translation or summarization)
generated_ids = predictions.predictions

# Decode the generated tokens to text
decoded_preds = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

# Print some sample predictions
for i in range(5):
    print(f"Predicted: {decoded_preds[i]}")
    print(f"Reference: {val_dataset[i]['target_text']}\n")  # Assuming 'target_text' is in the dataset


ValueError: No columns in the dataset match the model's forward method signature. The following columns have been ignored: [answer_start, answerable, lang, context, question, answer_inlang, answer]. Please check the dataset and model. You may need to set `remove_unused_columns=False` in `TrainingArguments`.

#### 2. No context model (Tom)

In [ ]:
import pandas as pd
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from transformers import Trainer, TrainingArguments
from datasets import Dataset
from nltk.translate.bleu_score import corpus_bleu
import nltk
import torch
from tqdm import tqdm

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

languages = ['ru', 'fi', 'ja']

# Download NLTK data files (if not already installed)
nltk.download('punkt')

# Initialize tokenizer and model
tokenizer = MT5Tokenizer.from_pretrained('google/mt5-small')
model = MT5ForConditionalGeneration.from_pretrained('google/mt5-small')

# Convert pandas dataframes to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df[train_df['lang'].isin(languages) & train_df['answer_inlang'].notna()])
val_dataset = Dataset.from_pandas(val_df[val_df['lang'].isin(languages) & val_df['answer_inlang'].notna()])

# Tokenization function
def tokenize_function(examples):
    # Tokenize the input (questions)
    inputs = tokenizer(
        examples['question'],
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    # Tokenize the target (answers)
    labels = tokenizer(
        examples['answer_inlang'],
        max_length=512,
        truncation=True,
        padding='max_length'
    )
    inputs['labels'] = labels['input_ids']
    return inputs

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=train_dataset.column_names)
tokenized_val = val_dataset.map(tokenize_function, batched=True, remove_columns=val_dataset.column_names)

# Set format for PyTorch tensors
tokenized_train.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])
tokenized_val.set_format(type='torch', columns=['input_ids', 'attention_mask', 'labels'])

# Define training arguments
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,                       # Adjust the number of epochs as needed
    per_device_train_batch_size=4,            # Adjust based on your GPU memory
    per_device_eval_batch_size=4,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_dir='./logs',
    logging_steps=10,
    save_total_limit=2,
    load_best_model_at_end=True,
    metric_for_best_model='eval_loss',
    greater_is_better=False,
)

# Create Trainer instance
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

# Train the model
trainer.train()

# Evaluation on the validation set
predictions = []
references = []

# Ensure the model is in evaluation mode
model.eval()

# Use GPU if available


for idx in tqdm(range(len(val_df))):
    question = val_df.iloc[idx]['question']
    answer = val_df.iloc[idx]['answer_inlang']

    # Tokenize the input question
    inputs = tokenizer(
        question,
        return_tensors='pt',
        truncation=True,
        max_length=512,
        padding='max_length'
    ).to(device)

    # Generate the output (prediction)
    with torch.no_grad():
        outputs = model.generate(
            input_ids=inputs['input_ids'],
            attention_mask=inputs['attention_mask'],
            max_length=512,
            num_beams=5,
            early_stopping=True,
        )

    # Decode the generated answer
    pred = tokenizer.decode(outputs[0], skip_special_tokens=True)
    predictions.append(nltk.word_tokenize(pred))
    references.append([nltk.word_tokenize(answer)])

# Calculate BLEU score
bleu_score = corpus_bleu(references, predictions)
print(f"BLEU score: {bleu_score}")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Map:   0%|          | 0/300 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:494: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


OutOfMemoryError: CUDA out of memory. Tried to allocate 12.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 3.06 MiB is free. Process 3558 has 14.74 GiB memory in use. Of the allocated memory 14.55 GiB is allocated by PyTorch, and 58.65 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
!pip install transformers datasets sacrebleu pandas evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 13.5 MB/s eta 0:00:00


In [ ]:
from transformers import MT5ForConditionalGeneration, MT5Tokenizer
from datasets import Dataset
import pandas as pd

# Load mT5-small model and tokenizer
model_name = 'google/mt5-small'
tokenizer = MT5Tokenizer.from_pretrained(model_name)
model = MT5ForConditionalGeneration.from_pretrained(model_name)

splits = {'train': 'train.parquet', 'validation': 'validation.parquet'}
# Train and validation sets
train_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["train"])
val_df = pd.read_parquet("hf://datasets/coastalcph/tydi_xor_rc/" + splits["validation"])

languages = ['ru', 'fi', 'ja']
train_dataset = Dataset.from_pandas(train_df[train_df['lang'].isin(languages) & train_df['answer_inlang'].notna()])
val_dataset = Dataset.from_pandas(val_df[val_df['lang'].isin(languages) & val_df['answer_inlang'].notna()])

def preprocess_data(examples):
    # Tokenize the questions (inputs)
    inputs = examples['question']
    targets = examples['answer_inlang']

    # Tokenize the inputs with padding and truncation
    model_inputs = tokenizer(inputs, max_length=128, truncation=True, padding="max_length")

    # Tokenize the target answers with padding and truncation
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length").input_ids

    # Replace padding token ID (which is usually 0) with -100, to ignore these tokens in the loss calculation
    labels = [[(label if label != tokenizer.pad_token_id else -100) for label in label_seq] for label_seq in labels]

    # Add labels to model inputs
    model_inputs["labels"] = labels

    return model_inputs

# Tokenize the datasets
train_dataset = train_dataset.map(preprocess_data, batched=True)
val_dataset = val_dataset.map(preprocess_data, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/82.0 [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/4.31M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/553 [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/1.20G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/300 [00:00<?, ? examples/s]

In [ ]:
# prompt: print the first 10 question and answer_inlang of the train_df dataframe

print(train_dataset[0])

{'question': 'ポーランドで農地改革が行われたことがある？', 'context': "As the Soviets advanced through Poland in 1944 and 1945, the German administration collapsed. The communist-controlled PKWN was installed in July 1944 in Lublin, the first major Polish city within the new boundaries to be seized by the Soviets from the Nazis, and began to take over the administration of the country as the Germans retreated. The Polish government in London formally protested the establishment of the PKWN. The PKWN was led by Edward Osóbka-Morawski, a socialist, and included other non-communists. The PKWN Manifesto was proclaimed in Chełm on July 22, initiating the crucial land reform. The agrarian reform, according to Norman Davies, was moderate and very popular. The communists constituted only a small, but highly organized and influential minority in the forming and gaining strength Polish pro-Soviet camp, which also included leaders and factions from such main political blocks as the agrarian, socialist, Zionist, and n

In [ ]:
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments

# Define training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir='./results_mt5',
    eval_strategy="no",
    save_strategy="no",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    num_train_epochs=3,
    logging_dir='./logs_mt5',
    save_total_limit=3,
    learning_rate=1e-5,
    logging_steps=1,
    fp16=False,  # Disable mixed precision to avoid potential issues
    report_to="none"  # Disable W&B logging
)

# Initialize the Trainer
trainer = Seq2SeqTrainer(
    model=model,                           # The mT5 model
    args=training_args,                    # The training arguments we set up
    train_dataset=train_dataset,           # The training dataset
    tokenizer=tokenizer,                   # The tokenizer
)

trainer.train()

Step,Training Loss
1,28.647900
2,31.595900
3,26.507900
4,28.264100
5,34.287900
6,25.206100
7,28.926300
8,33.986900
9,29.371000
10,28.272200


TrainOutput(global_step=30, training_loss=28.37010758717855, metrics={'train_runtime': 26.6958, 'train_samples_per_second': 16.857, 'train_steps_per_second': 1.124, 'total_flos': 59484340224000.0, 'train_loss': 28.37010758717855, 'epoch': 3.0})

In [ ]:
# Modify the predict call to include max_new_tokens to control length of generation
predictions = trainer.predict(val_dataset, max_new_tokens=50)

def safe_decode(predictions):
    try:
        return tokenizer.batch_decode(predictions, skip_special_tokens=True)
    except IndexError as e:
        print(f"Error decoding: {e}")
        return [""] * len(predictions)  # Return empty strings if there's a decoding error


# Modify the compute_bleu function to handle potential issues in token decoding
def compute_bleu(pred):
    labels_ids = pred.label_ids
    pred_ids = pred.predictions

    # Remove invalid token IDs (e.g., negative values)
    pred_ids = [[token for token in pred if token >= 0] for pred in pred_ids]

    # Safely decode predicted and reference texts
    pred_str = safe_decode(pred_ids)
    labels_str = safe_decode(labels_ids)

    # BLEU expects a list of references, where each reference itself is a list
    labels_str = [[label] for label in labels_str]

    # Compute BLEU score
    bleu = bleu_metric.compute(predictions=pred_str, references=labels_str)
    return {"bleu": bleu["score"]}


# Compute BLEU score manually after predictions
bleu_score = compute_bleu(predictions)
print(f"BLEU Score: {bleu_score['bleu']}")


Error decoding: piece id is out of range.
BLEU Score: 0.0


In [ ]:
print(predictions.predictions)

[[     0 250099      1 ...      0      0      0]
 [     0 250099    291 ...      0      0      0]
 [     0 250099    291 ...      0      0      0]
 ...
 [     0 250099      1 ...      0      0      0]
 [     0 250099      1 ...      0      0      0]
 [     0 250099      1 ...      0      0      0]]


In [ ]:
!pip install googletrans

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 4.7 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.0.0-py3-none-any.whl size=15719 sha256=15abe6be6aa6aedebf1ada3609049e1bf000b718149aacb3c77d2488be6dca0f
  Stored in directory: /root/.cache/pip/wheels/b3/81/ea/8b030407f8ebfc2f857814e086bb22ca2d4fea1a7be63652ab
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling chardet-5.2.0:
  

In [ ]:
from googletrans import Translator

# Initialize the translator
translator = Translator()

# Make predictions on the validation set
predictions = trainer.predict(val_dataset)

# Decode predicted answers
predicted_answers = tokenizer.batch_decode(predictions.predictions, skip_special_tokens=True)

# Decode the labels (ground truth answers) for comparison, if needed
true_answers = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# Iterate over the first 10 predictions
for i in range(10):
    # Get the predicted answer and translate it to English
    predicted_answer = predicted_answers[i]
    try:
        translation = translator.translate(predicted_answer, dest='en')
        predicted_answer_en = translation.text
    except Exception as e:
        predicted_answer_en = "Translation unavailable"
        print(f"Error translating prediction {i+1}: {e}")

    # Print the original question, predicted answer, and the translation
    print(f"Prediction {i+1}:")
    print(f"Predicted Answer: {predicted_answer}")
    print(f"Translated Answer (English): {predicted_answer_en}")
    print("-" * 80)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1258: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Error translating prediction 1: 'NoneType' object has no attribute 'group'
Prediction 1:
Predicted Answer: 
Translated Answer (English): Translation unavailable
--------------------------------------------------------------------------------
Error translating prediction 2: 'NoneType' object has no attribute 'group'
Prediction 2:
Predicted Answer: 
Translated Answer (English): Translation unavailable
--------------------------------------------------------------------------------
Error translating prediction 3: 'NoneType' object has no attribute 'group'
Prediction 3:
Predicted Answer: 
Translated Answer (English): Translation unavailable
--------------------------------------------------------------------------------
Error translating prediction 4: 'NoneType' object has no attribute 'group'
Prediction 4:
Predicted Answer: 
Translated Answer (English): Translation unavailable
--------------------------------------------------------------------------------
Error translating prediction 5: 

# Week 41+

While generating an answer is more flexible than extracting it as a span, it
may be right for the wrong reasons, i.e. the answer may be correct even if the
question is unanswerable given the context.
Use all questions in Finnish, Japanese and Russian to train (or fine-tune) a
model that receives the question and context as input and generates the English
answer. You can decide whether to train one model per question language or a
single model for all three languages.
Evaluate using a text generation metric on the validation set, and compare
the overall results between answerable and unanswerable examples. Can the
model answer correctly even when the answer is not provided in the context?
Discuss the results.

In [ ]:
!pip install datasets
!pip install Evaluate

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import evaluate
import torch

In [ ]:

# Filter for Finnish, Japanese, and Russian questions where 'answer' is available
train_df_filtered = train_df[train_df['lang'].isin(['fi', 'ja', 'ru']) & train_df['answer'].notna()]
val_df_filtered = val_df[val_df['lang'].isin(['fi', 'ja', 'ru']) & val_df['answer'].notna()]


# Load pre-trained model and tokenizer
model_name = "google/mt5-small"  # or any other suitable multilingual model
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


def preprocess_function(examples):
    inputs = [f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])]
    targets = examples["answer"]  # Generate the English answer
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

# Convert to Hugging Face Datasets
train_dataset = Dataset.from_pandas(train_df_filtered)
val_dataset = Dataset.from_pandas(val_df_filtered)

# Tokenize datasets
tokenized_train_dataset = train_dataset.map(preprocess_function, batched=True)
tokenized_val_dataset = val_dataset.map(preprocess_function, batched=True)


# Define training arguments
training_args = TrainingArguments(
    output_dir="./results_en_answers",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
)

# Train the model
trainer.train()

# Evaluate using BLEU or ROUGE metric
metric = evaluate.load("sacrebleu") # or "rouge"


# Function to make predictions
def generate_answers(examples):
    inputs = tokenizer([f"question: {q} context: {c}" for q, c in zip(examples["question"], examples["context"])],
                      return_tensors="pt", padding=True, truncation=True, max_length=512)
    # Move inputs to cuda after calling contiguous
    inputs = inputs.to("cuda")
    for key in inputs:
        inputs[key] = inputs[key].contiguous() # Ensure tensors are contiguous before moving to cuda
    outputs = model.generate(**inputs, max_length=128)
    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

# Generate predictions on validation set
val_dataset = val_dataset.map(lambda examples: {"predictions": generate_answers(examples)}, batched=True, batch_size=4)

# Compute metrics
results = metric.compute(predictions=val_dataset["predictions"], references=val_dataset["answer"])
print(results)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
You are using a model of type mt5 to instantiate a model of type t5. This is not supported for all configurations of models and can yield errors.


Map:   0%|          | 0/6410 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:4126: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1380 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,1.409200,0.535200


ValueError: You are trying to save a non contiguous tensor: `encoder.block.0.layer.0.SelfAttention.q.weight` which is not allowed. It either means you are trying to save tensors which are reference of each other in which case it's recommended to save only the full tensors, and reslice at load time, or simply call `.contiguous()` on your tensor to pack it before saving.